In [0]:
#Mount ADLS Gen 2

tiers = {"bronze", "silver", "gold"}
adls_paths = {tier: f"abfss://{tier}@datacesar.dfs.core.windows.net/" for tier in tiers}

print(adls_paths)

#Accessing paths
bronze_adls = adls_paths["bronze"]
silver_adls = adls_paths["silver"]
gold_adls = adls_paths["gold"]

dbutils.fs.ls(bronze_adls)
dbutils.fs.ls(silver_adls)
dbutils.fs.ls(gold_adls)

In [0]:
import requests
import json
from datetime import date, timedelta

In [0]:
# Get base parameters
dbutils.widgets.text("start_date", "")
dbutils.widgets.text("end_date", "")
start_date = dbutils.widgets.get("start_date")
end_date = dbutils.widgets.get("end_date")

In [0]:
#Construct the API URL with correct dates.
url = f"https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime={start_date}&endtime={end_date}"

In [0]:
url

In [0]:
try:
    #Get request for fetching data
    response = requests.get(url)

    #Verify if the request was successful
    response.raise_for_status() #Returns HTTPError for bad responses
    data = response.json().get('features', [])

    if not data :
        print("No data returned for the date range specified")
    else:
        #Specify the ADLS path
        file_path = f"{bronze_adls}/{start_date}_earthquake_data.json"

        #Save the json data
        json_data = json.dumps(data, indent=4)
        dbutils.fs.put(file_path, json_data, overwrite = True)
        print(f"Data successfully saved to {file_path}")
except requests.exceptions.RequestException as e:
    print(f"Error fetching data from API: {e}")

In [0]:
data[0]

In [0]:
# Define your variables
output_data = {
    "start_date": start_date,
    "end_date": end_date,
    "bronze_adls": bronze_adls,
    "silver_adls": silver_adls,
    "gold_adls": gold_adls
}

# Serialize the dictionary to a JSON string
output_json = json.dumps(output_data)

# Log the serialized JSON for debugging
print(f"Serialized JSON: {output_json}")

# Return the JSON string
dbutils.notebook.exit(output_json)